In [85]:
import codecs,math,re,glob

In [86]:
numLangue="01"
nbLots=3
nRepLot="/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Lots/"
nRepBrut="/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Grimm/"

In [147]:
def nomFichierLot(numLangue,numLot,charabia=True,lNgram=3):
    if charabia:
        return nRepLot+"Extraits-%s-Lot%d-Ngram%d-DecoupesEntro.txt"%(numLangue,numLot,lNgram)
    else:
        return nRepBrut+"Extraits-%s-Lot%d-brut.text"%(numLangue,numLot)

def getConte(numLangue,lots,charabia=True,lNgram=3):
    texte=[]
    filtrePonct=u":,;()\t-’´'`…–‚«»"+'"' # ici l'espace n'est pas filtré
    if not isinstance(lots, list):
        lots=[lots]
    for numLot in lots:
        with codecs.open(nomFichierLot(numLangue,numLot,charabia,lNgram),"r",encoding="utf8") as input:
            texteChaine="\n".join([l.strip() for l in input.readlines() if l.strip()!="" and l.strip()[0]!="#"])
        stops=re.escape(u".?!")+"\n"
        texte.extend([l.strip() for l in re.split(u"[%s]"%stops,texteChaine) if l!=""])
    if not charabia:
        tempTexte=[]
        for phrase in texte:
            tempPhrase=""
            for lettreBrute in phrase:
                if not lettreBrute in filtrePonct:
                    tempPhrase+=lettreBrute.lower()
#                    if lettreBrute in "?!":
#                        tempPhrase+=" "+lettreBrute
            if tempPhrase!="":
                tempTexte.append(tempPhrase)
        texte=[p.strip() for p in tempTexte]
    return texte

In [97]:
chunkedText=getConte("01",[1,2,3],charabia=True)
originalText=getConte("01",[1,2,3],charabia=False)

In [98]:
def getSpacesIndex(texte):
    index=[]
    nbSpaces=0
    for n,t in enumerate(texte):
        if t==" ":
            index.append(n-nbSpaces-1)
            nbSpaces+=1
    return index

In [114]:
resultats={}

In [176]:
lots=[1,2,3]
debug=0
lNgram=7
resultats[lNgram]={}
for numLangue in ["%02d"%(n+1) for n in range(11)]:
    print "Charabia",numLangue
    if not numLangue in  resultats[lNgram]:
        resultats[lNgram][numLangue]={}
    chunkedText=getConte(numLangue,lots,charabia=True,lNgram=lNgram)
    originalText=getConte(numLangue,lots,charabia=False)
    nbGoodSpaces=0
    nbPredictedSpaces=0
    nbGlobalSpaces=0
    nbGlobalLetters=0
    for n,l in enumerate(chunkedText):
        nbGlobalLetters+=len(originalText[n].replace(" ",""))-1
        nbGood=len(set(getSpacesIndex(chunkedText[n])) & set(getSpacesIndex(originalText[n])))
        nbGoodSpaces+=nbGood
        nbPredictions=len(set(getSpacesIndex(chunkedText[n])))
        nbPredictedSpaces+=nbPredictions
        nbSpaces=len(set(getSpacesIndex(originalText[n])))
        nbGlobalSpaces+=nbSpaces
        if nbPredictions>0:
            precision=float(nbGood)/nbPredictions
        if nbSpaces>0:
            recall=float(nbGood)/nbSpaces
        if numLangue=="00": 
            print n
            print l
            print originalText[n]
            print len(originalText[n].replace(" ",""))-1
            if nbPredictions>0 and nbSpaces>0:
                print "precision %2.2f, recall %2.2f"%(precision, recall)
    precision=float(nbGoodSpaces)/nbPredictedSpaces
    recall=float(nbGoodSpaces)/nbGlobalSpaces
    print "overall precision %2.2f, recall %2.2f"%(precision, recall)
    resultats[lNgram][numLangue]["precision"]=precision
    resultats[lNgram][numLangue]["recall"]=recall
    nbChunkedLetters=nbGlobalLetters-nbGlobalSpaces
    TP=nbGoodSpaces
    FP=nbPredictedSpaces-nbGoodSpaces
    FN=nbGlobalSpaces-nbGoodSpaces
    TN=nbChunkedLetters-(nbPredictedSpaces-nbGoodSpaces)
    resultats[lNgram][numLangue]["TP"]=TP
    resultats[lNgram][numLangue]["FP"]=FP
    resultats[lNgram][numLangue]["FN"]=FN
    resultats[lNgram][numLangue]["TN"]=TN
    print "TP %d, FP %d, TN %d, FN %d"%(resultats[lNgram][numLangue]["TP"],resultats[lNgram][numLangue]["FP"],
                                       resultats[lNgram][numLangue]["TN"],resultats[lNgram][numLangue]["FN"])
    FMeasure=(2*precision*recall/(precision+recall))
    print "F-Measure %2.2f"%FMeasure
    resultats[lNgram][numLangue]["FMeasure"]=FMeasure
    GMeasure=(math.sqrt(float(TP)/(TP+FP)*float(TP)/(TP+FN)))
    print "G-Measure %2.2f"%GMeasure
    resultats[lNgram][numLangue]["GMeasure"]=GMeasure
    

Charabia 01
overall precision 0.66, recall 0.29
TP 760, FP 391, TN 8366, FN 1826
F-Measure 0.41
G-Measure 0.44
Charabia 02
overall precision 0.47, recall 0.39
TP 735, FP 817, TN 10061, FN 1126
F-Measure 0.43
G-Measure 0.43
Charabia 03
overall precision 0.44, recall 0.52
TP 2421, FP 3132, TN 25171, FN 2237
F-Measure 0.47
G-Measure 0.48
Charabia 04
overall precision 0.69, recall 0.42
TP 2254, FP 1003, TN 16889, FN 3072
F-Measure 0.53
G-Measure 0.54
Charabia 05
overall precision 0.64, recall 0.44
TP 1728, FP 954, TN 13843, FN 2159
F-Measure 0.53
G-Measure 0.54
Charabia 06
overall precision 0.52, recall 0.43
TP 1968, FP 1806, TN 22643, FN 2614
F-Measure 0.47
G-Measure 0.47
Charabia 07
overall precision 0.49, recall 0.51
TP 1758, FP 1833, TN 18504, FN 1700
F-Measure 0.50
G-Measure 0.50
Charabia 08
overall precision 0.69, recall 0.36
TP 1111, FP 507, TN 9748, FN 2014
F-Measure 0.47
G-Measure 0.49
Charabia 09
overall precision 0.44, recall 0.47
TP 1464, FP 1827, TN 16889, FN 1626
F-Measure 0.

In [177]:
resultats

{2: {'01': {'FMeasure': 0.6715844475180575,
   'FN': 401,
   'FP': 1736,
   'GMeasure': 0.6861811127939208,
   'TN': 7021,
   'TP': 2185,
   'precision': 0.5572558020913032,
   'recall': 0.8449342614075793},
  '02': {'FMeasure': 0.4699577784995128,
   'FN': 414,
   'FP': 2850,
   'GMeasure': 0.5116967607414781,
   'TN': 8028,
   'TP': 1447,
   'precision': 0.3367465673725855,
   'recall': 0.7775389575497045},
  '03': {'FMeasure': 0.41551849166062366,
   'FN': 1220,
   'FP': 8452,
   'GMeasure': 0.46197152032357097,
   'TN': 19851,
   'TP': 3438,
   'precision': 0.2891505466778806,
   'recall': 0.738085015027909},
  '04': {'FMeasure': 0.6813533834586466,
   'FN': 795,
   'FP': 3443,
   'GMeasure': 0.695273001598768,
   'TN': 14449,
   'TP': 4531,
   'precision': 0.5682217205919238,
   'recall': 0.8507322568531731},
  '05': {'FMeasure': 0.6370442391832766,
   'FN': 611,
   'FP': 3122,
   'GMeasure': 0.6569230565277014,
   'TN': 11675,
   'TP': 3276,
   'precision': 0.5120350109409191,
  

In [178]:
def fMeasure(precision,recall):
    return 2*precision*recall/(precision+recall)

def getFMeasure(resultat):
    return fMeasure(resultat["precision"],resultat["recall"])

bestMeasure={l:{k:{"best":0,"lNgram":None} for k in ["precision","recall","FMeasure","GMeasure"]} for l in ["%02d"%(n+1) for n in range(11)]}
for l in resultats:
#    print "lNgram",l
    for n in resultats[l]:
#        print "Charabia",n
        for k in ["precision","recall","FMeasure","GMeasure"]:
#            print k
            if resultats[l][n][k]>bestMeasure[n][k]["best"]:
                bestMeasure[n][k]["best"]=resultats[l][n][k]
                bestMeasure[n][k]["lNgram"]=l
bestMeasure

{'01': {'FMeasure': {'best': 0.6916129032258064, 'lNgram': 3},
  'GMeasure': {'best': 0.7013203593066059, 'lNgram': 3},
  'precision': {'best': 0.6602953953084274, 'lNgram': 7},
  'recall': {'best': 0.8449342614075793, 'lNgram': 2}},
 '02': {'FMeasure': {'best': 0.5002575107296138, 'lNgram': 3},
  'GMeasure': {'best': 0.5364380094296364, 'lNgram': 3},
  'precision': {'best': 0.4735824742268041, 'lNgram': 7},
  'recall': {'best': 0.7829124126813541, 'lNgram': 3}},
 '03': {'FMeasure': {'best': 0.5121829224723116, 'lNgram': 5},
  'GMeasure': {'best': 0.5434562397133278, 'lNgram': 5},
  'precision': {'best': 0.4359805510534846, 'lNgram': 7},
  'recall': {'best': 0.8168741949334478, 'lNgram': 4}},
 '04': {'FMeasure': {'best': 0.7151898734177214, 'lNgram': 3},
  'GMeasure': {'best': 0.7242030937421314, 'lNgram': 3},
  'precision': {'best': 0.6920478968375806, 'lNgram': 7},
  'recall': {'best': 0.8507322568531731, 'lNgram': 2}},
 '05': {'FMeasure': {'best': 0.6838056680161942, 'lNgram': 3},
 

In [183]:
measure="GMeasure"
for ch in sorted(bestMeasure):
    prefix=""
    if bestMeasure[ch][measure]["lNgram"]!=3:
        prefix="\t"
    print ch,prefix+str(bestMeasure[ch][measure]["best"]), bestMeasure[ch][measure]["lNgram"]

01 0.701320359307 3
02 0.53643800943 3
03 	0.543456239713 5
04 0.724203093742 3
05 0.699890721487 3
06 0.532613769154 3
07 0.576536679474 3
08 0.731841440997 3
09 	0.550215225001 5
10 0.511217630816 3
11 0.709995476098 3


In [194]:
csv=[]
for l in resultats:
    for n in resultats[l]:
        csv.append('"%s";%d;%2.2f;%2.2f;%2.2f;%2.2f'%(n,l,resultats[l][n]["precision"],resultats[l][n]["recall"],resultats[l][n]["FMeasure"],resultats[l][n]["GMeasure"]))
csv=[u"Charabia;lNgram;précision;rappel;F-Mesure;G-Mesure"]+sorted(csv)        

In [195]:
for l in csv:
    print l

Charabia;lNgram;précision;rappel;F-Mesure;G-Mesure
"01";2;0.56;0.84;0.67;0.69
"01";3;0.59;0.83;0.69;0.70
"01";4;0.56;0.74;0.63;0.64
"01";5;0.58;0.64;0.61;0.61
"01";6;0.62;0.45;0.52;0.53
"01";7;0.66;0.29;0.41;0.44
"02";2;0.34;0.78;0.47;0.51
"02";3;0.37;0.78;0.50;0.54
"02";4;0.37;0.75;0.49;0.53
"02";5;0.39;0.67;0.49;0.51
"02";6;0.44;0.53;0.48;0.48
"02";7;0.47;0.39;0.43;0.43
"03";2;0.29;0.74;0.42;0.46
"03";3;0.35;0.80;0.49;0.53
"03";4;0.36;0.82;0.50;0.54
"03";5;0.38;0.77;0.51;0.54
"03";6;0.41;0.66;0.50;0.52
"03";7;0.44;0.52;0.47;0.48
"04";2;0.57;0.85;0.68;0.70
"04";3;0.62;0.85;0.72;0.72
"04";4;0.58;0.79;0.67;0.68
"04";5;0.61;0.73;0.66;0.67
"04";6;0.65;0.58;0.61;0.61
"04";7;0.69;0.42;0.53;0.54
"05";2;0.51;0.84;0.64;0.66
"05";3;0.56;0.87;0.68;0.70
"05";4;0.54;0.81;0.65;0.66
"05";5;0.56;0.74;0.64;0.65
"05";6;0.60;0.61;0.60;0.60
"05";7;0.64;0.44;0.53;0.54
"06";2;0.33;0.72;0.45;0.49
"06";3;0.38;0.74;0.50;0.53
"06";4;0.38;0.72;0.50;0.52
"06";5;0.41;0.65;0.50;0.51
"06";6;0.48;0.57;0.52;0.52
"06"